In [1]:
# 需要结合对模型的读取以及不同ratio数据的处理加入GMM
# 实现路径：1. 先读取model，做到可以计算结果
#2.再根据不同的给定的label情况，拟合GMM计算分类准确度， 输出图表
import sys
sys.path.append('..')
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
import code_package.tools_func as tool
import code_package.model_package as mcr2_model
DEVICE = 'cuda'
import os
import ssGMM
import pandas as pd
import importlib
importlib.reload(ssGMM)

<module 'ssGMM' from 'd:\\linan\\code_simulated_data\\LRD_project\\code_mnist\\ssGMM.py'>

In [2]:
# 重新读取数据
def cal_test_performance(data_set, mcr2, dim, set_name='test'):
    t_model_sensor_data = data_set[set_name][0].cuda()
    t_model_con_data = data_set[set_name][1].cuda()
    partial_label_input = data_set[set_name][2].cuda()
    true_label = data_set[set_name][3]
    mcr2.cuda()
    with torch.no_grad():
        res = mcr2(t_model_sensor_data, t_model_con_data, partial_label_input)
    loss = mcr2.loss_function(res,t_model_sensor_data)
    X = res['mu'].cpu().numpy()
    y = data_set['test'][3].cpu().numpy().copy() 
    # labels
    y[data_set['test'][2].sum(axis=1).cpu().numpy()==0]=-1
    ss = ssGMM.SemiSupervisedGMM(n_components=dim,random_state=0)
    ss.fit(X, y)   # y unlabeled = -1
    labels = ss.predict(X)
    acc = mcr2_model.cluster_acc(labels, data_set['test'][3].cpu().numpy())[0]
    res = {}
    res['loss'] = loss['loss'].item()
    res['reconstruction_loss'] = loss['recons_loss'].item()
    res['kl_loss'] = loss['kl_loss'].item()
    res['acc'] = acc
    return res

In [6]:
import yaml
res_cvae = {}
base_dir = './res_vae/'
for dim in [2,3,4,5,6,7,8,9,10]:
    latent_dim = dim
    class_dim = dim
    dim_dir = os.path.join(base_dir, 'latent_dim_'+str(latent_dim))
    for ratio in [0]:#, 1/4, 1]:
        ratio_dir = os.path.join(dim_dir, 'label_ratio_0')
        # load data
        data_dir =os.path.join('./dataloaders/', 'latent_dim_'+str(latent_dim), 'label_ratio_'+str(ratio))
        data_loader, data_set = torch.load(os.path.join(data_dir, 'dataset.pth'), map_location=DEVICE)
        # prepare model
        params_path = os.path.join('model_params', 'mnist_ae.yaml')
        mnist_params = yaml.load(open(params_path, 'r'), Loader=yaml.FullLoader)

        encoder_params = mnist_params['encoder_params']
        decoder_params = mnist_params['decoder_params']
        encoder_params['class_dim'] = class_dim
        encoder_params['latent_dim'] = latent_dim
        decoder_params['class_dim'] = class_dim
        decoder_params['latent_dim'] = latent_dim

        for kl_weights in [1e-3]:
            kl_dir = os.path.join(ratio_dir, 'kl_'+str(kl_weights))
            model_dir = os.path.join(kl_dir, 'models')
            ls = os.listdir(model_dir)
            ls.sort()
            val_loss = torch.load(os.path.join(kl_dir, 'val_loss.pth'), map_location=DEVICE)
            best_start = str(min(val_loss, key=val_loss.get))
            best_model_path = os.path.join(model_dir,'best_overall_model.pth')
            #best_res_path = os.path.join(model_dir, best_start, 'res_output.pth')
            #res = torch.load(best_res_path, weights_only=True,  map_location=DEVICE)
            state_dict = torch.load(best_model_path, weights_only=True, map_location=DEVICE)[0]
            mcr2 =  mcr2_model.CVAE(encoder_params, decoder_params, is_ts=0, kl_weights=kl_weights)
            mcr2.to(DEVICE)
            mcr2.load_state_dict(state_dict)
            mcr2.eval()
            output = cal_test_performance(data_set,mcr2, dim, set_name='test')
            res_cvae[kl_weights, ratio, dim] = output

C:\Users\linan\AppData\Local\Temp\ipykernel_36696\1441731404.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_loader, data_set = torch.load(os.path.join(data_dir, '

In [4]:
pd.DataFrame(res_cvae).T

loss  reconstruction_loss   kl_loss       acc
0.001 0 2   0.024510             0.019436  5.074435  0.573402
        3   0.029928             0.023517  6.410943  0.527785
        4   0.032635             0.025877  6.758019  0.620153
        5   0.031004             0.023121  7.883500  0.486111
        6   0.034292             0.026985  7.307861  0.510895
        7   0.032495             0.023941  8.553181  0.693610
        8   0.032092             0.023566  8.526052  0.633402
        9   0.033466             0.024986  8.480380  0.532236
        10  0.033354             0.024723  8.630737  0.677417

In [6]:
pd.DataFrame(res_cvae).T.to_csv('cvae_res_all.csv')